# Batch Predictions
![](img/batch_transform_tensorflow.gif)

In [14]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Setup Batch Transform Model

In [15]:
%store -r training_job_name

In [16]:
print(training_job_name)

tensorflow-training-2020-05-22-22-14-01-963


In [17]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2020-05-22-22-14-01-963/output/model.tar.gz to ./model.tar.gz


In [ ]:
!tar -xvzf ./model.tar.gz

In [ ]:
!saved_model_cli show --all --dir ./tensorflow/saved_model/0/

In [20]:
!pygmentize ./src_tsv/inference.py

import json
import tensorflow as tf
from transformers import DistilBertTokenizer

review_body_column_idx_tsv = 13

classes=[1, 2, 3, 4, 5]

max_seq_length=128

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def input_handler(data, context):
    transformed_instances = []
    print(type(data))
    print(data)

    for instance in data:
        print(type(instance))
        print(instance)

        data_str = instance.decode('utf-8')
        print(type(data_str))
        print(data_str)

        data_str_split = data_str.split('\t')
        print(len(data_str_split))
        if (len(data_str_split) >= review_body_column_idx_tsv):
            print(data_str_split[review_body_column_idx_tsv])

        tokens_a = tokenizer.tokenize(data_str_split[review_body_column_idx_tsv])

        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []  
        segmen

In [21]:
from sagemaker.tensorflow.serving import Model

batch_env = {
  'SAGEMAKER_TFS_DEFAULT_MODEL_NAME': 'saved_model',
  'SAGEMAKER_TFS_ENABLE_BATCHING': 'true',
  'SAGEMAKER_TFS_BATCH_TIMEOUT_MICROS': '50000', # microseconds
  'SAGEMAKER_TFS_MAX_BATCH_SIZE': '10000',
  'SAGEMAKER_MODEL_SERVER_TIMEOUT': '3600' # Seconds
}

batch_model = Model(entry_point='inference.py',
                    source_dir='src_tsv',       
                    model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
                    role=role,
                    framework_version='2.1.0',
                    env=batch_env)

To increase performance, you can increase the MaxConcurrentTransforms parameter.  Tune this on a single instance before trying to scale out the number of instances - especially if you have a small file count, the multiple instances can be a big waste.  Note that `max_concurrent_transforms * max_payload <= 100`

In [ ]:
batch_predictor = batch_model.transformer(strategy='MultiRecord', 
                                          instance_type='ml.c5.18xlarge',
                                          instance_count=1,
                                          accept='text/csv',
                                          assemble_with='Line',
                                          max_concurrent_transforms=1,
                                          max_payload=1, # This is in Megabytes (not number of records)
                                          env=batch_env)

# Specify Input Data

In [ ]:
predict_csv_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(predict_csv_s3_uri)

In [ ]:
!aws s3 ls --recursive $predict_csv_s3_uri

# Start Batch Predictions

In [ ]:
batch_predictor.transform(data=predict_csv_s3_uri,
                          split_type='Line',
                          compression_type='Gzip',
                          content_type='text/csv',
                          experiment_config=None,
                          wait=False)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}?region={}&tab=Monitor">Batch Prediction Job</a></b>'.format(region, batch_predictor.latest_transform_job.job_name, region)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TransformJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'.format(region, batch_predictor.latest_transform_job.job_name)))


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/s3/buckets/{}/{}/?region={}">Batch Prediction S3 Output</a></b>'.format(bucket, batch_predictor.latest_transform_job.job_name, region)))


In [ ]:
print('Waiting for batch prediction job: ' + batch_predictor.latest_transform_job.job_name)

batch_predictor.wait(logs=False)

# _Wait Until the ^^ Batch Transform Job ^^ Completes_

# Check Output Data

After the transform job has completed, download the output data from S3.

For each file in the input data, we have a corresponding file with a ".out" extension.  This .out file contains the predicted labels for each input row. 

In [ ]:
# Download the output data from S3 to local filesystem
batch_prediction_output_s3_uri = batch_predictor.output_path

In [ ]:
!aws s3 cp --recursive $batch_prediction_output_s3_uri/ batch_prediction_output/

In [ ]:
!ls batch_prediction_output/